In [1]:
# import from the files in this directory
from dotenv import load_dotenv
from classifier import *
from data_fetcher import *
from parser import *
from orchestrator import *
from logger_setup import *
import os
import json
import pandas as pd
import numpy as np
import re
import time

In [3]:
config_path = 'config_experiment.json'  # Config with input file details
load_dotenv()
orchestrator = Orchestrator(config_path)
orchestrator.setup_data_fetcher()

orchestrator.py - line 20 - INFO - Data_Gatherer Orchestrator initialized. Extraction step Model: gemini-2.0-flash-exp
orchestrator.py - line 45 - INFO - Data fetcher setup completed.


<selenium.webdriver.firefox.webdriver.WebDriver (session="99d25b7c-bd95-4def-9146-67ed89ce5285")>

In [6]:
df = pd.read_csv('exp_input/PRIDEid_HTML_data.csv')

In [7]:
urls = df['publication'].tolist()
print(f"Number of URLs: {len(urls)}")

Number of URLs: 236


In [8]:
results = {}
for i,url in enumerate(urls):
    time.sleep(15)
    orchestrator.current_url = url
    orchestrator.logger.info(f"Processing URL {i+1}: {orchestrator.current_url}")
    orchestrator.publisher = orchestrator.data_fetcher.url_to_publisher_domain(url)
    #orchestrator.logger.info("Using LLMParser to parse data.")
    orchestrator.parser = LLMParser(orchestrator.XML_config, orchestrator.logger)
    raw_data = df.loc[df['publication'] == url, 'raw_html'].iloc[0]
    parsed_data = orchestrator.parser.parse_data(raw_data, orchestrator.publisher, orchestrator.current_url, raw_data_format="full_HTML")
    parsed_data['source_url'] = url
    orchestrator.logger.info(f"Parsed data extraction completed. Elements collected: {len(parsed_data)}")
    results[url] = parsed_data

1839347971.py - line 5 - INFO - Processing URL 1: https://dx.doi.org/10.1001/JAMANEUROL.2024.4763
data_fetcher.py - line 34 - INFO - Publisher: doi
parser.py - line 26 - INFO - Parser initialized.
parser.py - line 154 - INFO - Function call: parse_data(api_data(<class 'str'>), publisher, current_url_address, additional_data, full_HTML)
parser.py - line 247 - INFO - Extracting links from full HTML content.
parser.py - line 293 - INFO - Function_call: normalize_full_DOM(api_data). Length of raw api data: 192677 tokens
parser.py - line 691 - INFO - Loading prompt: GEMINI_from_full_input_Examples_4
parser.py - line 706 - INFO - Prompt messages total length: 3262 tokens
parser.py - line 712 - INFO - Prompt ID: gemini-2.0-flash-exp-0-49dcd5511235e9fdd919a4f3e75213b67b9b5ce9ece3c367e2432f12c62875a5
parser.py - line 731 - INFO - Requesting datasets from content using model: gemini-2.0-flash-exp, temperature: 0, messages: messages
parser.py - line 830 - INFO - Found 1 candidates in the response

In [9]:
combined_df = pd.DataFrame()
for url, data in results.items():
    combined_df = pd.concat([combined_df, data], ignore_index=True)
    
combined_df.to_csv(orchestrator.config['full_output_file'], index=False)

In [10]:
combined_df

,dataset_identifier,data_repository,source_url,dataset_webpage
0,n/a,n/a,https://dx.doi.org/10.1001/JAMANEUROL.2024.4763,NaN
1,PXD055649,proteomecentral.proteomexchange.org,https://dx.doi.org/10.1002/CBIC.202400831,https://proteomecentral.proteomexchange.org/cg...
2,PXD060372,proteomecentral.proteomexchange.org,https://dx.doi.org/10.1002/CBIC.202400882,https://proteomecentral.proteomexchange.org/cg...
3,PXD028078,http://www.ebi.ac.uk/pride/archive/,https://dx.doi.org/10.1002/PRCA.202300107,https://www.ebi.ac.uk/pride/archive/projects/P...
4,PXD056865,www.ebi.ac.uk,https://dx.doi.org/10.1002/anie.202420149,https://www.ebi.ac.uk/pride/archive/projects/P...
...,...,...,...,...
568,TCGA Pan-Cancer dataset,UCSC Xena,https://www.ncbi.nlm.nih.gov/pubmed/39875995,na
569,PMC11790920,www.ncbi.nlm.nih.gov,https://www.ncbi.nlm.nih.gov/pubmed/39900909,na
570,PMC11799215,www.ncbi.nlm.nih.gov,https://www.ncbi.nlm.nih.gov/pubmed/39910101,na
571,10.1038/s41467-025-56648-6,doi.org,https://www.ncbi.nlm.nih.gov/pubmed/39910101,na


In [23]:
recall_list = []
def calculate_performance_metrics(orchestrator, ground_truth_df: pd.DataFrame, eval_df: pd.DataFrame) -> dict:
    """
    Calculate performance metrics (precision, recall, F1-score) for dataset extraction,
    considering exact, partial, and alias matches. Includes verbose debugging output.
    """

    
    # Initialize counters for total scores
    total_precision = 0
    total_recall = 0
    
    num_tot_sources = len(eval_df['source_url'].unique())
    num_sources = 0

    orchestrator.logger.info(f"Number of unique source pages: {num_sources}")

    # Iterate through each unique source page
    for source_page in eval_df['source_url'].unique():
        orchestrator.logger.info(f"\nStarting performance evaluation for source page: {source_page}")

        # Extract ground truth datasets for this source page
        gt_data = ground_truth_df[ground_truth_df['publication'] == source_page]
        
        # skip if no datasets in the HTML
        # if ground_truth_df[''] not in ground_truth_df['raw_html']:
        #     continue
        present = False
        for i, row in gt_data.iterrows():
            if row['dataset_uid'] in row['raw_html']:
                orchestrator.logger.info(f"Dataset id {row['dataset_uid']} reference in the raw_html for {source_page}")
                present = True
                break
        
        if not present:
            orchestrator.logger.info(f"No datasets references in the raw_html for {source_page}")
            continue
        
        num_sources += 1
        
        gt_datasets = set(gt_data['dataset_uid'].dropna().str.lower())
        gt_repositories = set(gt_data['repo_name'].dropna().str.lower())

        orchestrator.logger.info(f"Ground truth datasets: {gt_datasets}")
        # orchestrator.logger.info(f"Ground truth repositories: {gt_repositories}")

        # Extract evaluation datasets for this source page
        eval_data = eval_df[eval_df['source_url'] == source_page]
        eval_datasets = set(eval_data['dataset_identifier'].dropna().str.lower())
        # eval_webpages = set(eval_data['dataset_webpage'].dropna().str.lower())
        # eval_repositories = set(eval_data['data_repository'].dropna().str.lower())

        orchestrator.logger.info(f"Evaluation datasets: {eval_datasets}")
        # orchestrator.logger.info(f"Evaluation webpages: {eval_webpages}")
        # orchestrator.logger.info(f"Evaluation repositories: {eval_repositories}")

        # Handle cases where both ground truth and evaluation are empty
        if not gt_datasets and not eval_datasets:
            orchestrator.logger.info("No datasets in both ground truth and evaluation. Perfect precision and recall.")
            total_precision += 1
            total_recall += 1
            continue

        # Initialize match counters
        exact_matches = 0
        partial_matches = 0
        false_positives = 0
        false_negatives = 0

        # Track matches to avoid double counting
        matched_gt = set()
        matched_eval = set()

        # Check for exact matches first
        for eval_id in eval_datasets:
            if eval_id in gt_datasets:
                orchestrator.logger.info(f"Exact match found: {eval_id}")
                exact_matches += 1
                matched_gt.add(eval_id)
                matched_eval.add(eval_id)

        # Check for partial matches and aliases
        for eval_id in eval_datasets - matched_eval:
            for gt_id in gt_datasets - matched_gt:
                if eval_id in gt_id or gt_id in eval_id:  # Partial match or alias
                    orchestrator.logger.info(f"Partial or alias match found: eval_id={eval_id}, gt_id={gt_id}")
                    partial_matches += 1
                    matched_gt.add(gt_id)
                    matched_eval.add(eval_id)
                    break

        # Calculate False Positives (remaining unmatched eval datasets)
        FP = eval_datasets - matched_eval

        # Remove aliases from False Positives
        for eval_id in list(FP):
            for matched_id in matched_gt:
                if eval_id in matched_id or matched_id in eval_id:  # Alias detected
                    orchestrator.logger.info(f"Removing alias from false positives: eval_id={eval_id}, matched_id={matched_id}")
                    FP.discard(eval_id)
                    break

        false_positives = len(FP)
        orchestrator.logger.info(f"False positives: {FP}")

        # Calculate False Negatives (remaining unmatched ground truth datasets)
        FN = gt_datasets - matched_gt
        false_negatives = len(FN)
        orchestrator.logger.info(f"False negatives: {FN}")

        # Calculate precision and recall for this source page
        true_positives = exact_matches + partial_matches
        precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
        recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0

        orchestrator.logger.info(f"Precision for {source_page}: {precision}")
        orchestrator.logger.info(f"Recall for {source_page}: {recall}")
        
        if recall == 0:
            recall_list.append(source_page)

        # Accumulate totals
        total_precision += precision
        total_recall += recall

    # Calculate overall metrics
    average_precision = total_precision / num_sources if num_sources > 0 else 0
    average_recall = total_recall / num_sources if num_sources > 0 else 0
    f1_score = 2 * (average_precision * average_recall) / (average_precision + average_recall) if (average_precision + average_recall) > 0 else 0
    
    orchestrator.logger.info(f"\nPerformance evaluation completed for {num_sources} source pages.")

    return {
        "average_precision": average_precision,
        "average_recall": average_recall,
        "f1_score": f1_score
    }

In [24]:
calculate_performance_metrics(orchestrator, df, combined_df)

724350232.py - line 16 - INFO - Number of unique source pages: 0
724350232.py - line 20 - INFO - 
Starting performance evaluation for source page: https://dx.doi.org/10.1001/JAMANEUROL.2024.4763
724350232.py - line 36 - INFO - No datasets references in the raw_html for https://dx.doi.org/10.1001/JAMANEUROL.2024.4763
724350232.py - line 20 - INFO - 
Starting performance evaluation for source page: https://dx.doi.org/10.1002/CBIC.202400831
724350232.py - line 31 - INFO - Dataset id PXD055649 reference in the raw_html for https://dx.doi.org/10.1002/CBIC.202400831
724350232.py - line 44 - INFO - Ground truth datasets: {'pxd055649'}
724350232.py - line 53 - INFO - Evaluation datasets: {'pxd055649'}
724350232.py - line 77 - INFO - Exact match found: pxd055649
724350232.py - line 104 - INFO - False positives: set()
724350232.py - line 109 - INFO - False negatives: set()
724350232.py - line 116 - INFO - Precision for https://dx.doi.org/10.1002/CBIC.202400831: 1.0
724350232.py - line 117 - INFO

{'average_precision': 0.6576232169469874,
 'average_recall': 0.9918032786885246,
 'f1_score': 0.7908601740490612}